In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'extracting-attributes-from-fashion-images-jan-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F67553%2F7504710%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240318%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240318T170139Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D85222fac19d06bf53c29f4de05b9aac1f96118198c2f7cf9b02002a55c96870f9d03a8b34712e45b1aed56ead305ab751e287230d47d395d549a915fb1a98c5ff6332c5c83f8a3b4ebde3395aa1a6faf30401a289a82591f975906f5ad2622439d5876edf7877cb89b74e3276ace0f5add0cf9ac9442bcba3152f474d7d442a1a34868b90d242ab81ea3759762e7ce291e108ca667090d44528681d479f3d2fe66151a0923827e1a9499eecfb3c4619697a11b56318e7a56dfee850b224bef0497a8af75ee26081de1894aa488a5cd31eb3ce43502e6d180a300c7066b92f0a23c994d3eb7396ffdb59ff7333767b438ed2392db3a81a2ae252dd24a1baddfa2'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2


In [ ]:
# Load the data
train_data = pd.read_csv('/kaggle/input/extracting-attributes-from-fashion-images-jan-2024/train.csv')

# Prepare the data
X = train_data['file_name']  # Image file names
y = train_data['label']  # Class labels

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)


In [ ]:
# Define data augmentation and preprocessing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    horizontal_flip=True
)

In [ ]:
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
# Convert numeric labels to strings
train_data['label'] = train_data['label'].astype(str)

# Create the train generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    x_col='file_name',
    y_col='label',
    directory='/kaggle/input/extracting-attributes-from-fashion-images-jan-2024/train',
    shuffle=True,
    target_size=(28, 28),
    batch_size=32,
    class_mode='sparse'
)


In [ ]:
val_generator = val_datagen.flow_from_dataframe(
    dataframe=train_data,
    x_col="file_name",
    y_col="label",
    directory='/kaggle/input/extracting-attributes-from-fashion-images-jan-2024/train',
    subset="validation",  # Use the validation subset of the data
    shuffle=True,
    target_size=(28, 28),  # Update the target size if needed
    batch_size=32,
    class_mode='sparse'
)

In [ ]:
# Define the CNN model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30
)

In [ ]:
import cv2
abs_train_path = '/kaggle/input/extracting-attributes-from-fashion-images-jan-2024/train.csv'
abs_train_directory = '/kaggle/input/extracting-attributes-from-fashion-images-jan-2024/train'
abs_test_path = '/kaggle/input/extracting-attributes-from-fashion-images-jan-2024/sample_submission.csv'
abs_test_directory = '/kaggle/input/extracting-attributes-from-fashion-images-jan-2024/test'

df2 = pd.read_csv(abs_test_path)

X_test = []

# Resize and convert to grayscale for the test set
for i in range(df2.shape[0]):
    img = cv2.imread(os.path.join(abs_test_directory, df2.iloc[i]['file_name']), cv2.IMREAD_GRAYSCALE)

    img_resized = cv2.resize(img, (28, 28))

    # Convert to RGB format by duplicating the single channel
    img_resized = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
    if i%100 ==0:
        print(i)
    X_test.append(img_resized)
    if i % 100 == 0:
        print(i)

X_test_np = np.array(X_test)

In [ ]:
predictions = model.predict(X_test_np)
predicted_labels = tf.argmax(predictions, axis=1).numpy()

In [ ]:
import pandas as pd

# Create a DataFrame with the file_name and label columns
df_output = pd.DataFrame({'file_name': df2["file_name"], 'label': predicted_labels})

# Save the DataFrame to a CSV file
df_output.to_csv('output21.csv', index=False)